# Final Project: [HighwayEnv](https://github.com/Farama-Foundation/HighwayEnv/tree/master)

Ressources:
- **Highway-env** [👨‍💻Repo](https://github.com/Farama-Foundation/HighwayEnv/tree/master) | [📜Documentation](http://highway-env.farama.org/quickstart/)
- **OpenAI Gym**
- **Stable-Baselines3**: [👨‍💻Repo](https://github.com/DLR-RM/stable-baselines3) | [📜Documentation](https://stable-baselines.readthedocs.io/en/master/)

### Your task: Solve the Highway
![](https://raw.githubusercontent.com/eleurent/highway-env/gh-media/docs/media/highway.gif?raw=true)
- By Group of two, three
- Use *at least* two different RL Algorithms
  - try to implement at least one 'by hand'

### Evaluation
*Based on the report (showing that you understood what you did), the performances and the code (you did something that works).*

- **Produce a notebook**
  -  The notebook must run one one go, I will not loose time trying to fix your env...
  - Possible to send a git repo with the weight so that I ca nrun them locally.
- **Produce a 2-5 pages report**
  - Describe Your choices and explain the algorithms used.
  - Benchmark and compare them depending on their hyperparameters.

*Analysis could include exploration of hyperparameters, figures of training, explainations of how your algorithm works*

### Roadmap
- 📆 **12 feb 2025**: Send Me your group names and composition
- 📆 **25 mars** : Send a report (5-10 pages) and a notebook / script



## Utlilities
⚠️ *Do not Modify anything here !*

but always read everything to be sure of what is available

### Imports

In [1]:
!pip install gymnasium>=1.0.0a2
!pip install farama-notifications>=0.0.1
!pip install numpy>=1.21.0
!pip install pygame>=2.0.2
!pip install stable-baselines3[extra]
!pip install highway_env
#tensorboard loading if you want to use it
%load_ext tensorboard

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


### Utils

In [7]:
### VIDEO RECORDER
# Set up fake display; otherwise rendering will fail
import os
import base64
from pathlib import Path
from IPython import display as ipythondisplay
from tqdm import tqdm

os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=500, prefix="", video_folder="videos/", fps = 60):
    """
    :param env_id: (str)
    :param model: (RL model)
    :param video_length: (int)
    :param prefix: (str)
    :param video_folder: (str)
    """
    eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])
    eval_env.metadata["render_fps"] = fps
    # Start the video at step=0 and record 500 steps
    eval_env = VecVideoRecorder(
        eval_env,
        video_folder=video_folder,
        record_video_trigger=lambda step: step == 0,
        video_length=video_length,
        name_prefix=prefix,
    )
    obs = eval_env.reset()
    for _ in tqdm(range(video_length)):
        action, _ = model.predict(obs)
        obs, _, _, _ = eval_env.step(action)

    # Close the video recorder
    eval_env.close()

def show_videos(video_path="", prefix=""):
    """
    Taken from https://github.com/eleurent/highway-env

    :param video_path: (str) Path to the folder containing videos
    :param prefix: (str) Filter the video, showing only the only starting with this prefix
    """
    html = []
    for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay
                    loop controls style="height: 200px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [8]:
# prompt: define an evaluation function computing mean reward and elapsed episode time on a few runs of vectorized environments
import numpy as np

def evaluate(model, num_episodes=30):
    """
    Evaluates a reinforcement learning agent.

    Args:
        model: The trained RL model.
        env: The environment to evaluate the model on.
        num_episodes: The number of episodes to run for evaluation.

    Returns:
        A tuple containing the mean reward and the mean elapsed time per episode.
    """
    env_id = "highway-fast-v0"
    env = make_vec_env(env_id)
    episode_rewards = []
    episode_times = []
    print(f"evaluating Model on {num_episodes} episodes ...")
    for _ in tqdm(range(num_episodes)):
        obs = env.reset()
        done = False
        total_reward = 0
        start_time = 0 # Assuming env provides time information. Replace with actual time tracking
        current_time = 0

        while not done:
          action, _states = model.predict(obs, deterministic=True)
          obs, reward, done, info = env.step(action)
          total_reward += reward
          current_time += 1 # Replace with actual elapsed time from env info

        episode_rewards.append(total_reward)
        episode_times.append(current_time - start_time)

    mean_reward = np.mean(episode_rewards)
    mean_time = np.mean(episode_times)
    std_reward = np.std(episode_rewards)
    std_time = np.std(episode_times)
    print(f"\n{'-'*50}\nResults :\n\t- Mean Reward: {mean_reward:.3f} ± {std_reward:.2f} \n\t- Mean elapsed Time per episode: {mean_time:.3f} ± {std_time:.2f}\n{'-'*50}")
    return mean_reward, mean_time


## The Highway Environment

In [9]:
## IMPORTS
import gymnasium as gym
from stable_baselines3 import PPO, DQN
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv
import highway_env  # noqa: F401

## Load and explore Environment
Lets first load an untrained model and see how it behaves in the environment.

In [10]:
env_id = "highway-fast-v0"
env = make_vec_env(env_id)
#instanciate model
model = PPO("MlpPolicy", env, verbose=1)

#generate video of random model
record_video(env_id, model, video_length=50, prefix="random-agent", fps = 5)
show_videos("videos", prefix="random-agent")

Using cpu device


 96%|██████████████████████████████████████████████████████████████████████████████▋   | 48/50 [00:02<00:00, 22.73it/s]

Saving video to E:\AI\projets\projetfinalRL\vf\videos\random-agent-step-0-to-step-50.mp4
Moviepy - Building video E:\AI\projets\projetfinalRL\vf\videos\random-agent-step-0-to-step-50.mp4.
Moviepy - Writing video E:\AI\projets\projetfinalRL\vf\videos\random-agent-step-0-to-step-50.mp4




100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:02<00:00, 21.45it/s]

Moviepy - Done !
Moviepy - video ready E:\AI\projets\projetfinalRL\vf\videos\random-agent-step-0-to-step-50.mp4


In [11]:
evaluate(model)

evaluating Model on 30 episodes ...


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:07<00:00,  3.79it/s]


--------------------------------------------------
Results :
	- Mean Reward: 6.616 ± 3.68 
	- Mean elapsed Time per episode: 7.767 ± 3.84
--------------------------------------------------


(6.6164837, 7.766666666666667)

Let's now explore the environments settings:
### Action Space
👉 Look at the action space, what actions can the model do ?

In [12]:
######### YOUR CODE HERE #########
print(f"Action space: {env.action_space}")

Action space: Discrete(5)


### Observation Space
👉 Look at the [documentation](http://highway-env.farama.org/observations/) for possibles observations of the agents on the Highway

👉 Look at the observation spae in our case

In [13]:
######### YOUR CODE HERE #########
print(f"Observation space: {env.observation_space}")

Observation space: Box(-inf, inf, (5, 5), float32)


# Training an Agent on the Environment
👉 **Now it is your turn**, train your agents
Recall:
- you must try and compare different RL Algorithms
- part of your grade will be the evaluation of your best Agent.

🔥Tips
- Use tensorboard to monitor your trainings
- install it locally to get faster and longer trainings (not mandatory, colab should be ok)

In [14]:
# if you wnat to use tensorboard, highly recommended
%tensorboard --logdir "highway"

In [17]:
# Install sb3-contrib pour TRPO, QRDQN et ARS
!pip install sb3-contrib

# Importation des bibliothèques nécessaires
import os
import time
import torch
from stable_baselines3 import PPO, DQN, A2C
from sb3_contrib import TRPO, QRDQN, ARS
from stable_baselines3.common.callbacks import EvalCallback, BaseCallback


# Créer un dossier pour les logs TensorBoard
log_dir = "./logs/"

# Définir l'environnement
env_id = "highway-fast-v0"
env = make_vec_env(env_id, n_envs=4)  # Utiliser 4 environnements pour un entraînement plus rapide

        
# Fonction d'entraînement avec tqdm
def train_model(model_class, env, model_name, total_timesteps, **kwargs):
    model_log_dir = os.path.join(log_dir, model_name)
    os.makedirs(model_log_dir, exist_ok=True)

    kwargs.setdefault('tensorboard_log', model_log_dir)
    kwargs.setdefault('policy', 'MlpPolicy')

    model = model_class(env=env, verbose=1,**kwargs)

    eval_callback = EvalCallback(
        env,
        best_model_save_path=os.path.join(model_log_dir, "best_model"),
        log_path=os.path.join(model_log_dir, "eval"),
        eval_freq=5000,
        deterministic=True,
        render=False,
    )

    # Lancer l'entraînement avec le callback tqdm
    model.learn(total_timesteps=total_timesteps, callback=eval_callback)

    model.save(os.path.join(model_log_dir, f"{model_name}_final"))

    return model



Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [18]:
# Entraînement des 6 modèles
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

# Vérifier si CUDA est disponible et définir le device
device = "cuda" if torch.cuda.is_available() else "cpu"

total_timesteps = 100

# DQN (Deep Q-Network):
# - buffer_size=15000 : Un buffer relativement modéré pour le DQN, adapté pour un environnement avec une dynamique d'exploration plus lente. Un buffer trop grand pourrait ralentir l'entraînement.
# - learning_rate=5e-4 : Une valeur classique pour DQN, offrant un bon compromis entre la stabilité et la vitesse de convergence.
# - gamma=0.8 : Un facteur de discount légèrement plus faible pour encourager un apprentissage à plus court terme. Cela peut être utile pour un environnement nécessitant des actions rapides.
# - batch_size=32 : Une taille de batch standard, permettant de collecter suffisamment de données tout en restant efficace en termes de temps de calcul.
# - target_update_interval=50 : Le modèle met à jour la cible toutes les 50 itérations, un bon compromis pour maintenir la stabilité tout en restant réactif.
# - device="cuda" : L'entraînement sur GPU est activé, ce qui permet d'accélérer considérablement le processus d'entraînement pour de grandes tailles de réseau.
dqn_model = train_model(
    model_class=DQN,
    env=env,
    model_name="dqn_highway",
    total_timesteps=total_timesteps,
    policy_kwargs=dict(net_arch=[256, 256]),
    learning_rate=5e-4,
    buffer_size=15000,
    learning_starts=200,
    batch_size=32,
    gamma=0.8,
    train_freq=1,
    gradient_steps=1,
    target_update_interval=50,
    device=device
)


# PPO (Proximal Policy Optimization):
# - learning_rate=3e-4 : Un learning rate modéré pour garantir une bonne stabilité pendant l'apprentissage.
# - n_steps=2048 : Le nombre d'étapes de collecte de données par itération d'entraînement, assez grand pour un bon échantillonnage, mais sans surcharger la mémoire.
# - batch_size=64 : La taille du batch est ajustée pour maximiser l'efficacité de l'entraînement tout en assurant une bonne convergence.
# - gamma=0.99 : Un facteur de discount classique pour un apprentissage plus long terme, favorisant des actions qui maximisent la récompense future.
# - gae_lambda=0.95 : L'utilisation de GAE pour mieux estimer les avantages, avec un lambda modéré pour un compromis entre biais et variance.
# - clip_range=0.2 : Le clip de la politique pour éviter les mises à jour trop agressives. Cela permet de conserver une bonne stabilité.
# - vf_coef=0.5 : La pondération de la fonction de valeur dans la fonction de perte, équilibrant la politique et la valeur.
ppo_model = train_model(
    model_class=PPO,
    env=env,
    model_name="ppo_highway",
    total_timesteps=total_timesteps,
    policy_kwargs=dict(net_arch=[256, 256]),
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.2,
    ent_coef=0.0,
    vf_coef=0.5,
    max_grad_norm=0.5,
    device=device
)


# A2C (Advantage Actor-Critic):
# - learning_rate=3e-4 : Un taux d'apprentissage modéré pour un apprentissage stable tout en évitant les oscillations.
# - n_steps=5 : Le nombre d'étapes par itération. Utiliser une petite valeur pour réduire la mémoire et garder un calcul rapide, tout en restant précis.
# - gamma=0.99 : Un facteur de discount classique, assurant une prise en compte des récompenses futures sur le long terme.
# - vf_coef=0.5 : Un poids équilibré pour la fonction de valeur dans la perte, permettant de maintenir l'équilibre entre la politique et la fonction de valeur.
# - max_grad_norm=0.5 : Une normalisation des gradients pour éviter les gradients explosifs et assurer un apprentissage plus stable.
a2c_model = train_model(
    model_class=A2C,
    env=env,
    model_name="a2c_highway",
    total_timesteps=total_timesteps,
    policy_kwargs=dict(net_arch=[256, 256]),
    learning_rate=3e-4,
    n_steps=5,
    gamma=0.99,
    gae_lambda=0.95,
    ent_coef=0.0,
    vf_coef=0.5,
    max_grad_norm=0.5,
    device=device
)

# QR-DQN (Quantile Regression DQN):
# - learning_rate=1e-3 : Un taux d'apprentissage standard pour QR-DQN, permettant une mise à jour stable des Q-values.
# - buffer_size=1000000 : Un grand buffer pour stocker suffisamment de transitions et favoriser une exploration riche.
# - batch_size=256 : Une taille de batch standard pour QR-DQN, permettant de calculer des gradients plus stables.
# - gamma=0.99 : Un facteur de discount qui prend en compte les récompenses futures à long terme.
qrdqn_model = train_model(
    model_class=QRDQN,
    env=env,
    model_name="qrdqn_highway",
    total_timesteps=total_timesteps,
    policy_kwargs=dict(net_arch=[256, 256]),
    learning_rate=1e-3,
    buffer_size=1000000,
    learning_starts=500,
    batch_size=256,
    gamma=0.99,
    train_freq=1,
    gradient_steps=1,
    device=device
)

# TRPO (Trust Region Policy Optimization):
# - learning_rate=1e-3 : Un taux d'apprentissage lent pour une mise à jour stable et contrôlée de la politique.
# - gamma=0.99 : Un facteur de discount classique qui favorise les récompenses futures.
trpo_model = train_model(
    model_class=TRPO,
    env=env,
    model_name="trpo_highway",
    total_timesteps=total_timesteps,
    policy_kwargs=dict(net_arch=[256, 256]),
    learning_rate=1e-3,
    gamma=0.99,
    device=device
)


# Augmented Random Search (ARS):
# - learning_rate=3e-4 : Un taux d'apprentissage modéré pour garantir une bonne stabilité pendant l'apprentissage.
# - n_steps=2048 : Le nombre d'étapes de collecte de données par itération d'entraînement, assez grand pour un bon échantillonnage, mais sans surcharger la mémoire.
# - batch_size=64 : La taille du batch est ajustée pour maximiser l'efficacité de l'entraînement tout en assurant une bonne convergence.
# - gamma=0.99 : Un facteur de discount classique pour un apprentissage plus long terme, favorisant des actions qui maximisent la récompense future.
# - gae_lambda=0.95 : L'utilisation de GAE pour mieux estimer les avantages, avec un lambda modéré pour un compromis entre biais et variance.
ars_model = train_model(
    model_class=ARS,
    env=env,
    model_name="ars_highway",
    total_timesteps=total_timesteps,
    policy_kwargs=dict(net_arch=[256, 256]),  # La politique pour ARS peut ne pas être nécessairement un MLP, mais un réseau simple pour l'optimisation des paramètres.
    learning_rate=3e-4,
    device=device
)



# Sauvegarder les modèles finaux
ppo_model.save("highway_ppo_final")
dqn_model.save("highway_dqn_final")
a2c_model.save("highway_a2c_final")
qrdqn_model.save("highway_qrdqn_final")
trpo_model.save("highway_trpo_final")
ars_model.save("highway_ars_final")

print("Entraînement terminé ! Modèles sauvegardés.")


Using cpu device
Logging to ./logs/dqn_highway\DQN_1
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 7.75     |
|    ep_rew_mean      | 6.09     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 25       |
|    time_elapsed     | 1        |
|    total_timesteps  | 48       |
----------------------------------
Using cpu device
Logging to ./logs/ppo_highway\PPO_1


KeyboardInterrupt: 

In [18]:
# Comparaison des modèles
models = {
    "DQN": dqn_model,
    "PPO": ppo_model,
    "A2C": a2c_model,
    "QR-DQN": qrdqn_model,
    "TRPO": trpo_model,
    "ARS": ars_model
}

# Évaluation et affichage des résultats pour tous les modèles
for model_name, model in models.items():
    print(f"\nÉvaluation du modèle {model_name}...")
    model_reward = evaluate(model)
    print(f"Récompense totale du modèle {model_name} : {model_reward}")

# Comparer les performances (Récompenses)
print("\nComparaison des modèles :")
for model_name, model in models.items():
    model_reward = evaluate(model)
    print(f"{model_name} : {model_reward}")


Évaluation du modèle DQN...
evaluating Model on 30 episodes ...


  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [00:12<00:00,  2.44it/s]



--------------------------------------------------
Results :
	- Mean Reward: 11.354 ± 6.77 
	- Mean elapsed Time per episode: 14.567 ± 8.16
--------------------------------------------------
Récompense totale du modèle DQN : (np.float32(11.354009), np.float64(14.566666666666666))

Évaluation du modèle PPO...
evaluating Model on 30 episodes ...


100%|██████████| 30/30 [00:24<00:00,  1.22it/s]


...


--------------------------------------------------
Results :
	- Mean Reward: 20.322 ± 1.98 
	- Mean elapsed Time per episode: 29.500 ± 2.69
--------------------------------------------------
QR-DQN : (np.float32(20.322254), np.float64(29.5))
evaluating Model on 30 episodes ...


100%|██████████| 30/30 [00:24<00:00,  1.22it/s]



--------------------------------------------------
Results :
	- Mean Reward: 20.514 ± 1.78 
	- Mean elapsed Time per episode: 29.567 ± 2.33
--------------------------------------------------
TRPO : (np.float32(20.513552), np.float64(29.566666666666666))
evaluating Model on 30 episodes ...


100%|██████████| 30/30 [00:24<00:00,  1.21it/s]


--------------------------------------------------
Results :
	- Mean Reward: 21.187 ± 0.82 
	- Mean elapsed Time per episode: 30.000 ± 0.00
--------------------------------------------------
ARS : (np.float32(21.186886), np.float64(30.0))


In [19]:
### SAVE YOUR FINAL MODEL
model_final = qrdqn_model #YOUR MODEL
model_final.save("highway_final")

# Evalutation
⚠️ *Do not Modify anything here !*

Now that your Agents are trained, we evaluate them

In [20]:
evaluate(model_final)

evaluating Model on 30 episodes ...


  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [00:24<00:00,  1.24it/s]


--------------------------------------------------
Results :
	- Mean Reward: 20.112 ± 2.98 
	- Mean elapsed Time per episode: 29.233 ± 4.13
--------------------------------------------------


(np.float32(20.112238), np.float64(29.233333333333334))

In [21]:
env_id = "highway-v0"
# Generate video of trained model
record_video(env_id, model_final, video_length=70, prefix="trained-agent", fps = 5)
show_videos("videos", prefix="trained-agent")

 99%|█████████▊| 69/70 [00:23<00:00,  3.20it/s]

Saving video to c:\Users\Tom-Hugues\Desktop\Master EFREI\M2\S9\4 - Reinforcement learning\Projet\videos\trained-agent-step-0-to-step-70.mp4
MoviePy - Building video c:\Users\Tom-Hugues\Desktop\Master EFREI\M2\S9\4 - Reinforcement learning\Projet\videos\trained-agent-step-0-to-step-70.mp4.
MoviePy - Writing video c:\Users\Tom-Hugues\Desktop\Master EFREI\M2\S9\4 - Reinforcement learning\Projet\videos\trained-agent-step-0-to-step-70.mp4



100%|██████████| 70/70 [00:23<00:00,  2.96it/s]

MoviePy - Done !
MoviePy - video ready c:\Users\Tom-Hugues\Desktop\Master EFREI\M2\S9\4 - Reinforcement learning\Projet\videos\trained-agent-step-0-to-step-70.mp4


# 🎁 Bonus
If it was too easy for your, you can also try to train an agent on an even more difficult environment, for instance the `racetrack` *(see the highway env repo for other possible environments)*

---
![](https://raw.githubusercontent.com/eleurent/highway-env/gh-media/docs/media/racetrack-env.gif?raw=true)
